# Description

TODO/UPDATE: It computes gene enrichment on *all* the clustering results obtained using some correlation measure on GTEx v8 (specified under `Settings` below).

# Modules loading

In [1]:
import re
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed

import numpy as np
import pandas as pd
from tqdm import tqdm

from clustermatch import conf
from clustermatch.gene_enrich import run_enrich

# Settings

In [2]:
DATASET_CONFIG = conf.GTEX

In [3]:
CORRELATION_METHOD_NAME = "spearman_abs"

In [4]:
# GENE_SELECTION_STRATEGY = "var_pc_log2"

In [5]:
# clusterProfiler settings
ENRICH_FUNCTION = "enrichKEGG"
ENRICH_PARAMS = "hsa"

# Paths

In [6]:
INPUT_DIR = DATASET_CONFIG["CLUSTERING_DIR"]
display(INPUT_DIR)
assert INPUT_DIR.exists()

PosixPath('/opt/data/results/gtex_v8/clustering')

In [7]:
# this directory has the input data given to the clustering methods
SIMILARITY_MATRICES_DIR = DATASET_CONFIG["SIMILARITY_MATRICES_DIR"]
display(SIMILARITY_MATRICES_DIR)

PosixPath('/opt/data/results/gtex_v8/similarity_matrices')

In [8]:
SIMILARITY_MATRIX_FILENAME_TEMPLATE = DATASET_CONFIG[
    "SIMILARITY_MATRIX_FILENAME_TEMPLATE"
]
display(SIMILARITY_MATRIX_FILENAME_TEMPLATE)

'gtex_v8_data_{tissue}-{gene_sel_strategy}-{corr_method}.pkl'

In [9]:
OUTPUT_DIR = DATASET_CONFIG["GENE_ENRICHMENT_DIR"]
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
display(OUTPUT_DIR)

PosixPath('/opt/data/results/gtex_v8/gene_set_enrichment')

# Get data files

In [10]:
filename_pattern = re.compile(DATASET_CONFIG["CLUSTERING_FILENAME_PATTERN"])

In [11]:
# get input data files according to Settings
input_files = sorted(
    [
        f
        for f in INPUT_DIR.iterdir()
        if (m := re.search(filename_pattern, str(f))) is not None
        and m.group("corr_method") == CORRELATION_METHOD_NAME
    ]
)
display(len(input_files))
display(input_files[:5])

assert len(input_files) > 0

5

[PosixPath('/opt/data/results/gtex_v8/clustering/gtex_v8_data_adipose_subcutaneous-var_pc_log2-spearman_abs-SpectralClustering.pkl'),
 PosixPath('/opt/data/results/gtex_v8/clustering/gtex_v8_data_artery_tibial-var_pc_log2-spearman_abs-SpectralClustering.pkl'),
 PosixPath('/opt/data/results/gtex_v8/clustering/gtex_v8_data_muscle_skeletal-var_pc_log2-spearman_abs-SpectralClustering.pkl'),
 PosixPath('/opt/data/results/gtex_v8/clustering/gtex_v8_data_skin_sun_exposed_lower_leg-var_pc_log2-spearman_abs-SpectralClustering.pkl'),
 PosixPath('/opt/data/results/gtex_v8/clustering/gtex_v8_data_whole_blood-var_pc_log2-spearman_abs-SpectralClustering.pkl')]

# Preview input data

## Clustering results

In [12]:
tmp = pd.read_pickle(input_files[0])

In [13]:
tmp.shape

(25, 4)

In [14]:
tmp.head()

,params,partition,n_clusters,si_score
id,,,,
SpectralClustering #0,"{""affinity"": ""precomputed"", ""assign_labels"": ""...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, ...",2,0.097958
SpectralClustering #1,"{""affinity"": ""precomputed"", ""assign_labels"": ""...","[1, 3, 0, 4, 1, 1, 2, 1, 1, 1, 4, 0, 4, 0, 3, ...",5,0.085897
SpectralClustering #2,"{""affinity"": ""precomputed"", ""assign_labels"": ""...","[2, 8, 1, 9, 7, 2, 2, 2, 7, 2, 9, 8, 9, 4, 7, ...",10,0.100657
SpectralClustering #3,"{""affinity"": ""precomputed"", ""assign_labels"": ""...","[2, 13, 10, 5, 3, 2, 2, 2, 3, 2, 1, 1, 5, 3, 8...",15,0.103594
SpectralClustering #4,"{""affinity"": ""precomputed"", ""assign_labels"": ""...","[5, 1, 17, 4, 9, 5, 9, 5, 9, 5, 4, 15, 4, 9, 3...",20,0.097459


## Similarity matrices (input to clustering methods)

In [15]:
similarity_matrix_filename = SIMILARITY_MATRIX_FILENAME_TEMPLATE.format(
    tissue="adipose_subcutaneous",
    gene_sel_strategy="var_pc_log2",
    corr_method=CORRELATION_METHOD_NAME.split("_")[0]
    if not CORRELATION_METHOD_NAME.startswith("clustermatch")
    else CORRELATION_METHOD_NAME,
)
display(similarity_matrix_filename)

'gtex_v8_data_adipose_subcutaneous-var_pc_log2-spearman.pkl'

In [16]:
tmp = pd.read_pickle(SIMILARITY_MATRICES_DIR / similarity_matrix_filename)

In [17]:
tmp.shape

(5000, 5000)

In [18]:
tmp.head()

gene_ens_id,ENSG00000129824.15,ENSG00000149968.11,ENSG00000134184.12,ENSG00000224114.1,ENSG00000173432.10,ENSG00000067048.16,ENSG00000229807.10,ENSG00000012817.15,ENSG00000134339.8,ENSG00000175084.11,...,ENSG00000197467.13,ENSG00000279807.1,ENSG00000107742.12,ENSG00000176454.13,ENSG00000212206.1,ENSG00000104611.11,ENSG00000181800.5,ENSG00000171049.8,ENSG00000257671.1,ENSG00000180448.10
gene_ens_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000129824.15,1.000000,0.404501,0.016874,0.102808,-0.463379,0.710166,-0.649073,0.730501,-0.479843,0.279987,...,0.074034,-0.085351,-0.209944,0.212915,-0.106794,-0.030385,0.048907,-0.077775,-0.000937,-0.093243
ENSG00000149968.11,0.404501,1.000000,-0.062548,0.063588,-0.064418,0.286312,-0.333030,0.341573,-0.122310,0.153661,...,-0.166098,-0.207672,-0.135442,-0.103594,-0.197815,0.077693,0.177712,0.148204,0.188003,-0.083599
ENSG00000134184.12,0.016874,-0.062548,1.000000,0.035498,-0.064229,-0.017118,0.053330,0.023757,-0.085841,0.012899,...,0.167310,0.035928,0.001913,0.098022,-0.038101,0.083429,0.064641,-0.140293,0.011012,0.027150
ENSG00000224114.1,0.102808,0.063588,0.035498,1.000000,-0.009197,0.030979,-0.026245,0.021601,-0.018495,0.025221,...,-0.059515,0.038625,-0.011811,0.026184,-0.072435,0.000165,0.028716,0.094844,0.001404,-0.014584
ENSG00000173432.10,-0.463379,-0.064418,-0.064229,-0.009197,1.000000,-0.459325,0.344925,-0.454130,0.966881,-0.294052,...,-0.276992,-0.046945,0.274067,-0.423291,0.036829,0.115715,-0.076436,0.239778,0.033209,0.108790


### Convert Ensembl Gene IDs to Entrez IDs

In [19]:
input_filename = conf.GTEX["DATA_DIR"] / "gtex_entrez_gene_ids_mappings.pkl"
display(input_filename)
assert input_filename.exists()

PosixPath('/opt/data/data/gtex_v8/gtex_entrez_gene_ids_mappings.pkl')

In [20]:
gene_ids_mappings = pd.read_pickle(input_filename)

In [21]:
gene_ids_mappings.shape

(34527, 3)

In [22]:
gene_ids_mappings.head()

,gene_ens_id_v,ensembl_id,entrez_id
0,ENSG00000145309.5,ENSG00000145309,85438
1,ENSG00000175820.3,ENSG00000175820,643677
2,ENSG00000083454.21,ENSG00000083454,5026
3,ENSG00000071794.15,ENSG00000071794,6596
4,ENSG00000211918.1,ENSG00000211918,28503


In [23]:
gene_id_maps = gene_ids_mappings.set_index("gene_ens_id_v")["entrez_id"].to_dict()

In [24]:
dict(list(gene_id_maps.items())[0:2])

{'ENSG00000145309.5': '85438', 'ENSG00000175820.3': '643677'}

In [25]:
# is map from ensembl to entrez unique?
_tmp_index = [gene_id_maps[x] for x in tmp.index if x in gene_id_maps]
display(len(_tmp_index))
display(_tmp_index[:5])

4201

['6192', '4314', '2944', '100271063', '6288']

# Run

In [26]:
n_partitions_per_file = pd.read_pickle(input_files[0]).shape[0]
display(n_partitions_per_file)

25

In [27]:
# the number of tasks is the number of input files times number of partitions per file
n_tasks = len(input_files) * n_partitions_per_file
n_tasks = int(n_tasks)
display(f"number of tasks: {n_tasks}")

with ProcessPoolExecutor(max_workers=conf.GENERAL["N_JOBS"]) as executor, tqdm(
    total=n_tasks, ncols=100
) as pbar:
    for clustering_filepath in input_files:
        # extract from input clustering filename some sections, such as tissue name, etc
        m = re.search(filename_pattern, str(clustering_filepath.name))

        tissue = m.group("tissue")
        gene_sel_strategy = m.group("gene_sel_strategy")
        corr_method = m.group("corr_method")

        # update pbar description
        pbar.set_description(f"{tissue}/{gene_sel_strategy}")
        #         pbar.set_description(f"{corr_method}")

        # create output filepath template
        full_output_filename_template = (
            f"{clustering_filepath.stem}-{ENRICH_FUNCTION}-{ENRICH_PARAMS}.pkl"
        )

        # read clustering results
        clustering_df = pd.read_pickle(clustering_filepath)

        # get partitions' numbers
        tmp_partition = clustering_df.iloc[0].partition
        n_genes = tmp_partition.shape[0]
        n_clusters = np.unique(tmp_partition).shape[0]

        # use those sections to read the list of genes from the input data
        # file that the clustering algorithm received
        similarity_matrix_filename = SIMILARITY_MATRIX_FILENAME_TEMPLATE.format(
            tissue=tissue,
            gene_sel_strategy=gene_sel_strategy,
            corr_method=corr_method.split("_")[0]
            if not corr_method.startswith("clustermatch")
            else corr_method,
        )

        # get the universe of genes
        all_gene_ens_ids = pd.read_pickle(
            SIMILARITY_MATRICES_DIR / similarity_matrix_filename
        ).index.tolist()

        # convert gene ensembl ids to entrez and create clustering partition mask
        partition_mask = []
        all_gene_ids = []
        entrez_ids_added = set()  # this is faster

        for x in all_gene_ens_ids:
            if x not in gene_id_maps:
                partition_mask.append(False)
                continue

            new_entrez_id = gene_id_maps[x]

            # TODO: maybe this avoiding of repeated gene ids is not necessary?
            # do not add repeated ids
            if new_entrez_id in entrez_ids_added:
                partition_mask.append(False)
                continue

            all_gene_ids.append(new_entrez_id)
            entrez_ids_added.add(new_entrez_id)
            partition_mask.append(True)

        partition_mask = np.array(partition_mask, dtype=bool)
        all_gene_ids = np.array(all_gene_ids)
        assert np.unique(all_gene_ids).shape[0] == all_gene_ids.shape[0]
        assert all_gene_ids.shape[0] == np.sum(partition_mask)

        # iterate over clustering solutions (partitions) and GO ontologies
        futures = [
            executor.submit(
                run_enrich,
                all_gene_ids,
                "ENTREZID",
                cr.partition[partition_mask],
                ENRICH_FUNCTION,
                ENRICH_PARAMS,
            )
            for cr_idx, cr in clustering_df.sort_values("n_clusters").iterrows()
        ]

        # collect results
        results_full = []

        for task in as_completed(futures):
            task_results = task.result()

            # continue if no enrichment found
            if len(task_results) == 0:
                pbar.update(1)
                continue

            results_full.append(task_results[0])

            pbar.update(1)

        if len(results_full) == 0:
            # no significant results, continue
            continue

        # merge and serve
        pbar.set_description(f"{tissue}/{gene_sel_strategy}/saving")

        # full
        results_full_df = pd.concat(results_full, ignore_index=True).sort_values(
            ["n_clusters", "pvalue_adjust"]
        )

        results_full_df.to_pickle(
            OUTPUT_DIR
            / f"{clustering_filepath.stem}-{ENRICH_FUNCTION}-{ENRICH_PARAMS}.pkl",
        )

'number of tasks: 125'

whole_blood/var_pc_log2/saving: 100%|█████████████████████████████| 125/125 [05:12<00:00,  2.50s/it]
